In [1]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4

In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer

In [3]:
data=load_dataset('nyu-mll/glue','mnli')
data

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/392702 [00:00<?, ? examples/s]

Generating validation_matched split:   0%|          | 0/9815 [00:00<?, ? examples/s]

Generating validation_mismatched split:   0%|          | 0/9832 [00:00<?, ? examples/s]

Generating test_matched split:   0%|          | 0/9796 [00:00<?, ? examples/s]

Generating test_mismatched split:   0%|          | 0/9847 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 392702
    })
    validation_matched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9815
    })
    validation_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9832
    })
    test_matched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9796
    })
    test_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9847
    })
})

In [4]:
fast_tokenizer=AutoTokenizer.from_pretrained('bert-base-uncased')
slow_tokenizer=AutoTokenizer.from_pretrained('bert-base-uncased',use_fast=False)

def tokenizer_fast(example):
  return fast_tokenizer(example['premise'],example['hypothesis'],truncation=True)
def tokenizer_slow(example):
  return slow_tokenizer(example['premise'],example['hypothesis'],truncation=True)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
%time fast_output_batched=data.map(tokenizer_fast,batched=True)
%time fast_output=data.map(tokenizer_fast,batched=False)
%time slow_output_batched=data.map(tokenizer_slow,batched=True)
%time slow_output=data.map(tokenizer_slow,batched=False)

Map:   0%|          | 0/392702 [00:00<?, ? examples/s]

CPU times: user 1min 35s, sys: 892 ms, total: 1min 36s
Wall time: 1min 5s


Map:   0%|          | 0/392702 [00:00<?, ? examples/s]

Map:   0%|          | 0/9815 [00:00<?, ? examples/s]

Map:   0%|          | 0/9832 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/9796 [00:00<?, ? examples/s]

Map:   0%|          | 0/9847 [00:00<?, ? examples/s]

CPU times: user 5min 42s, sys: 1.7 s, total: 5min 44s
Wall time: 5min 47s


Map:   0%|          | 0/392702 [00:00<?, ? examples/s]

KeyboardInterrupt: 

In [5]:
context="North Korea says Sylvain fired a new weapon - is it lying?"
tokenizer=AutoTokenizer.from_pretrained('bert-base-uncased')
encoding=tokenizer(context,return_offsets_mapping=True)
print(type(encoding))

<class 'transformers.tokenization_utils_base.BatchEncoding'>


In [6]:
tokenizer.is_fast

True

In [7]:
encoding.is_fast

True

In [8]:
print(f"Tokens: {encoding.tokens()}")
print(f"Word index: {encoding.word_ids()}")
print(f"OffsetsL {encoding['offset_mapping']}")

Tokens: ['[CLS]', 'north', 'korea', 'says', 'sy', '##lva', '##in', 'fired', 'a', 'new', 'weapon', '-', 'is', 'it', 'lying', '?', '[SEP]']
Word index: [None, 0, 1, 2, 3, 3, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, None]
OffsetsL [(0, 0), (0, 5), (6, 11), (12, 16), (17, 19), (19, 22), (22, 24), (25, 30), (31, 32), (33, 36), (37, 43), (44, 45), (46, 48), (49, 51), (52, 57), (57, 58), (0, 0)]


In [9]:
start,end=encoding.word_to_chars(3)
context[start:end]

'Sylvain'

In [10]:
from transformers import pipeline
token_classifier=pipeline('token-classification')
token_classifier('Is Elon Musk worth his $56bn Tesla pay package?')

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/998 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

[{'entity': 'I-PER',
  'score': 0.9991874,
  'index': 2,
  'word': 'El',
  'start': 3,
  'end': 5},
 {'entity': 'I-PER',
  'score': 0.9981781,
  'index': 3,
  'word': '##on',
  'start': 5,
  'end': 7},
 {'entity': 'I-PER',
  'score': 0.99866164,
  'index': 4,
  'word': 'Mu',
  'start': 8,
  'end': 10},
 {'entity': 'I-PER',
  'score': 0.9896422,
  'index': 5,
  'word': '##sk',
  'start': 10,
  'end': 12},
 {'entity': 'I-MISC',
  'score': 0.8069328,
  'index': 12,
  'word': 'Te',
  'start': 29,
  'end': 31},
 {'entity': 'I-ORG',
  'score': 0.58452654,
  'index': 13,
  'word': '##sla',
  'start': 31,
  'end': 34}]

In [11]:
from transformers import pipeline

token_classifier = pipeline("token-classification", aggregation_strategy="simple")
token_classifier("Is Elon Musk worth his $56bn TESLA pay package?")

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'entity_group': 'PER',
  'score': 0.9958929,
  'word': 'Elon Musk',
  'start': 3,
  'end': 12},
 {'entity_group': 'ORG',
  'score': 0.96618986,
  'word': 'TESLA',
  'start': 29,
  'end': 34}]

In [17]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
checkpoint="dbmdz/bert-large-cased-finetuned-conll03-english"
tokenizer=AutoTokenizer.from_pretrained(checkpoint)
model=AutoModelForTokenClassification.from_pretrained(checkpoint)
inputs=tokenizer("Is Elon Musk worth his $56bn TESLA pay package?",return_tensors="pt")
outputs=model(**inputs)

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [15]:
inputs['input_ids']

tensor([[  101,  2181,  2896,  1320, 19569,  5276,  3869,  1117,   109,  4376,
          1830,  1179,   157,  9919, 10783,  2653,  7305,   136,   102]])

In [16]:
print(inputs['input_ids'].shape)
print(outputs.logits.shape)

torch.Size([1, 19])
torch.Size([1, 19, 9])


In [ ]:
inputs

{'input_ids': tensor([[  101,  2181,  2896,  1320, 19569,  5276,  3869,  1117,   109,  4376,
          1830,  1179,   157,  9919, 10783,  2653,  7305,   136,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [18]:
import torch
probability=torch.nn.functional.softmax(outputs.logits,dim=-1)[0].tolist()
prediction=torch.argmax(outputs.logits,dim=-1)[0].tolist()
#print(f"Probability: {probability}")
print(f"Prediction: {prediction}")

Prediction: [0, 0, 4, 4, 4, 4, 0, 0, 0, 0, 0, 0, 6, 6, 6, 0, 0, 0, 0]


In [19]:
model.config.id2label

{0: 'O',
 1: 'B-MISC',
 2: 'I-MISC',
 3: 'B-PER',
 4: 'I-PER',
 5: 'B-ORG',
 6: 'I-ORG',
 7: 'B-LOC',
 8: 'I-LOC'}

In [25]:
tokens=inputs.tokens()
result=[]

for ind,pred in enumerate(prediction):
  label=model.config.id2label[pred]
  if label!='O':
    result.append({'entity':label,'word':tokens[ind],"probability":probability[ind][pred]})
result

[{'entity': 'I-PER', 'word': 'El', 'probability': 0.9990755319595337},
 {'entity': 'I-PER', 'word': '##on', 'probability': 0.9978513717651367},
 {'entity': 'I-PER', 'word': 'Mu', 'probability': 0.9984338879585266},
 {'entity': 'I-PER', 'word': '##sk', 'probability': 0.9882110953330994},
 {'entity': 'I-ORG', 'word': 'T', 'probability': 0.9426470994949341},
 {'entity': 'I-ORG', 'word': '##ES', 'probability': 0.9737718105316162},
 {'entity': 'I-ORG', 'word': '##LA', 'probability': 0.9821505546569824}]

In [27]:
inputs_with_offsets=tokenizer("Is Elon Musk worth his $56bn TESLA pay package?",return_offsets_mapping=True)
inputs_with_offsets['offset_mapping']

[(0, 0),
 (0, 2),
 (3, 5),
 (5, 7),
 (8, 10),
 (10, 12),
 (13, 18),
 (19, 22),
 (23, 24),
 (24, 26),
 (26, 27),
 (27, 28),
 (29, 30),
 (30, 32),
 (32, 34),
 (35, 38),
 (39, 46),
 (46, 47),
 (0, 0)]

In [28]:
inputs_with_offsets

{'input_ids': [101, 2181, 2896, 1320, 19569, 5276, 3869, 1117, 109, 4376, 1830, 1179, 157, 9919, 10783, 2653, 7305, 136, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'offset_mapping': [(0, 0), (0, 2), (3, 5), (5, 7), (8, 10), (10, 12), (13, 18), (19, 22), (23, 24), (24, 26), (26, 27), (27, 28), (29, 30), (30, 32), (32, 34), (35, 38), (39, 46), (46, 47), (0, 0)]}

In [35]:
results = []
#inputs_with_offsets = tokenizer("", return_offsets_mapping=True)
tokens = inputs_with_offsets.tokens()
offsets = inputs_with_offsets["offset_mapping"]

for idx, pred in enumerate(prediction):
    label = model.config.id2label[pred]
    if label != "O":
        start, end = offsets[idx]
        results.append(
            {
                "entity": label,
                "score": probability[idx][pred],
                "word": tokens[idx],
                "start": start,
                "end": end,
            }
        )

result

[{'entity': 'I-PER', 'word': 'El', 'probability': 0.9990755319595337},
 {'entity': 'I-PER', 'word': '##on', 'probability': 0.9978513717651367},
 {'entity': 'I-PER', 'word': 'Mu', 'probability': 0.9984338879585266},
 {'entity': 'I-PER', 'word': '##sk', 'probability': 0.9882110953330994},
 {'entity': 'I-ORG', 'word': 'T', 'probability': 0.9426470994949341},
 {'entity': 'I-ORG', 'word': '##ES', 'probability': 0.9737718105316162},
 {'entity': 'I-ORG', 'word': '##LA', 'probability': 0.9821505546569824}]